In [ ]:
%cd /content/drive/My Drive/New/Object-Detection-API-master

/content/drive/My Drive/New/Object-Detection-API-master


In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 86.4MB 72kB/s 
     |████████████████████████████████| 28.7MB 155kB/s 
     |████████████████████████████████| 3.8MB 53.5MB/s 
     |████████████████████████████████| 450kB 49.6MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=f52bb29093b4d409d4644412c179cfcd326c86bbb31316a389913afe804e95c3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.0.3 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.5 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Succe

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -O weights/yolov3.weights

--2020-10-16 06:32:22--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘weights/yolov3.weights’

weights/yolov3.weig 100%[===================>] 236.52M   141KB/s    in 35m 48s 

2020-10-16 07:08:11 (113 KB/s) - ‘weights/yolov3.weights’ saved [248007048/248007048]



In [ ]:
!python load_weights.py

2020-10-16 14:11:52.942385: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-10-16 14:11:52.947001: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2020-10-16 14:11:52.947320: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1acebc0 executing computations on platform Host. Devices:
2020-10-16 14:11:52.947475: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): Host, Default Version
Model: "yolov3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, None,  0                                            
__________________________________________________________________________________________________
yolo_darknet (Model)

In [1]:
%cd /content/drive/My Drive/New/Object-Detection-API-master

/content/drive/My Drive/New/Object-Detection-API-master


In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2021-03-09 13:58:22--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.222.240.112, 52.22.246.104, 18.205.129.29, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.222.240.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.4’

ngrok-stable-linux- 100%[===================>]  13.13M  12.4MB/s    in 1.1s    

2021-03-09 13:58:23 (12.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.4’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [4]:
from IPython import get_ipython

In [5]:
!pip install flask

In [6]:
import flask
from IPython import get_ipython

In [7]:
get_ipython().system_raw('python app.py &')

In [8]:
get_ipython().system_raw('./ngrok http 5000 &')

In [9]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://0122df6e2e4b.ngrok.io


In [13]:

!pip install flask_ngrok


from flask_ngrok import run_with_ngrok

In [ ]:
import time
from absl import app, logging
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs
from flask import Flask, request, Response, jsonify, send_from_directory, abort
import os

# customize your API through the following parameters
classes_path = './data/labels/coco.names'
weights_path = './weights/yolov3.tf'
tiny = False                    # set to True if using a Yolov3 Tiny model
size = 416                      # size images are resized to for model
output_path = './detections/'   # path to output folder where images with detections are saved
num_classes = 80                # number of classes in model

# load in weights and classes
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

if tiny:
    yolo = YoloV3Tiny(classes=num_classes)
else:
    yolo = YoloV3(classes=num_classes)

yolo.load_weights(weights_path).expect_partial()
print('weights loaded')

class_names = [c.strip() for c in open(classes_path).readlines()]
print('classes loaded')

# Initialize Flask application
app = Flask(__name__)

# API that returns JSON with classes found in images


run_with_ngrok(app)  
@app.route('/detections', methods=['POST'])
def get_detections():
    raw_images = []
    images = request.files.getlist("images")
    image_names = []
    for image in images:
        image_name = image.filename
        image_names.append(image_name)
        image.save(os.path.join(os.getcwd(), image_name))
        img_raw = tf.image.decode_image(
            open(image_name, 'rb').read(), channels=3)
        raw_images.append(img_raw)
        
    num = 0
    
    # create list for final response
    response = []

    for j in range(len(raw_images)):
        # create list of responses for current image
        responses = []
        raw_img = raw_images[j]
        num+=1
        img = tf.expand_dims(raw_img, 0)
        img = transform_images(img, size)

        t1 = time.time()
        boxes, scores, classes, nums = yolo(img)
        t2 = time.time()
        print('time: {}'.format(t2 - t1))

        print('detections:')
        for i in range(nums[0]):
            print('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                            np.array(scores[0][i]),
                                            np.array(boxes[0][i])))
            responses.append({
                "class": class_names[int(classes[0][i])],
                "confidence": float("{0:.2f}".format(np.array(scores[0][i])*100))
            })
        response.append({
            "image": image_names[j],
            "detections": responses
        })
        img = cv2.cvtColor(raw_img.numpy(), cv2.COLOR_RGB2BGR)
        img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
        cv2.imwrite(output_path + 'detection' + str(num) + '.jpg', img)
        print('output saved to: {}'.format(output_path + 'detection' + str(num) + '.jpg'))

    #remove temporary images
    for name in image_names:
        os.remove(name)
    try:
        return jsonify({"response":response}), 200
    except FileNotFoundError:
        abort(404)

# API that returns image with detections on it
@app.route('/image', methods= ['POST'])
def get_image():
    image = request.files["images"]
    image_name = image.filename
    image.save(os.path.join(os.getcwd(), image_name))
    img_raw = tf.image.decode_image(
        open(image_name, 'rb').read(), channels=3)
    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, size)

    t1 = time.time()
    boxes, scores, classes, nums = yolo(img)
    t2 = time.time()
    print('time: {}'.format(t2 - t1))

    print('detections:')
    for i in range(nums[0]):
        print('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                        np.array(scores[0][i]),
                                        np.array(boxes[0][i])))
    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
    cv2.imwrite(output_path + 'detection.jpg', img)
    print('output saved to: {}'.format(output_path + 'detection.jpg'))
    
    # prepare image for response
    _, img_encoded = cv2.imencode('.png', img)
    response = img_encoded.tostring()
    
    #remove temporary image
    os.remove(image_name)

    try:
        return Response(response=response, status=200, mimetype='image/png')
    except FileNotFoundError:
        abort(404)
if __name__ == '__main__':
    app.run()
    #debug=True, host = '0.0.0.0', port=5000

weights loaded
classes loaded
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://cf577bc0d699.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Mar/2021 14:08:22] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2021 14:09:59] "GET /detections HTTP/1.1" 405 -
127.0.0.1 - - [09/Mar/2021 14:10:11] "POST /detections HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 14:12:40] "POST /img HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2021 14:12:48] "GET /img HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2021 14:13:13] "POST /img HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2021 14:13:38] "POST /img HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2021 14:14:42] "POST /image HTTP/1.1" 400 -
127.0.0.1 - - [09/Mar/2021 14:15:08] "POST /detection HTTP/1.1" 404 -
127.0.0.1 - - [09/Mar/2021 14:15:21] "POST /detections HTTP/1.1" 200 -
127.0.0.1 - - [09/Mar/2021 14:16:27] "POST /image HTTP/1.1" 400 -
127.0.0.1 - - [09/Mar/2021 14:16:32] "GET /image HTTP/1.1" 405 -


time: 2.011751651763916
detections:


...

output saved to: ./detections/detection.jpg
time: 1.6730430126190186
detections:
	horse, 0.9950188994407654, [0.20350093 0.0464673  0.9779596  0.943118  ]
	horse, 0.9284881353378296, [0.02258644 0.04818207 0.53483367 0.91256887]


127.0.0.1 - - [09/Mar/2021 14:18:37] "POST /detections HTTP/1.1" 200 -


output saved to: ./detections/detection1.jpg


127.0.0.1 - - [09/Mar/2021 14:19:06] "POST /image HTTP/1.1" 200 -


time: 1.6626403331756592
detections:
	horse, 0.9950188994407654, [0.20350093 0.0464673  0.9779596  0.943118  ]
	horse, 0.9284881353378296, [0.02258644 0.04818207 0.53483367 0.91256887]
output saved to: ./detections/detection.jpg


In [ ]:
!python app.py

2020-10-20 17:39:36.576925: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-20 17:39:38.925246: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-20 17:39:38.935491: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-20 17:39:38.935551: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (18ec181b2b38): /proc/driver/nvidia/version does not exist
2020-10-20 17:39:38.958088: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-10-